# CA2: Genetic Algorithm (EP)
## Amir Ali Vahidi 810199511

##### import needed libraries for further uses

In [1]:
import random
import bisect
from itertools import accumulate
from colorama import Fore ,Back ,Style

#### Initial values for calculations

In [2]:
# assign numbers to variables below
crossoverProbability = 0.5  # probabality
carryPercentage = 0.2  # prob
populationSize = 1000  # size
mutateProbability = 0.3

#### Calculating Fitness function
The fittest chromosomes have higher probability to be selected for the next 
generation. So I calculate absolute diffrence between every chromosome number representation and the goal number and one is divided by this result for the rankings. To avoid divide by zero problem, the value of diff is added by 1 for the division.

In [3]:
def calcFitness(chromosome):
    # define the fitness measure here
    equation_result = eval(chromosome)
    diff = abs(equation_result - goalNumber)
    return 1 / (diff + 1)

#### MatingPool:
The probability for each chromosomes is formulated by: $$P[i] = Fitness[i] / (Total Fitness Sum)$$
we calculate this probabilities and base on the result,we compute the cumulative probability values.\
Then, we randomly choose a number in range of (0,1) and select a new chromosome from the cumulative probability list to add to the MatingPool.(as much as cumulative probability is higher,the chance for a new chromosome to be in this cumulative probability is higher)\

#### Crossover function:
If random number generated is greater than crossover probability, then this chromosome from MatingPool will be added to the crossoverPool. but if not, we will randomly choose two population index and a division index where we break the those two populations and mix them based on the division index we randomly get.

#### Mutation function:
My code will mutate at most a fourth of the population and change operands or operators from a chromosome.


In [4]:
class EquationBuilder:

    def __init__(self, operators, operands, equationLength, goalNumber):
        self.operators = operators
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber

        # Create the earliest population at the begining
        self.population = self.makeFirstPopulation()

    def makeFirstPopulation(self):
        # TODO create random chromosomes to build the early population, and return it
        chromosomes = []
        for _ in range(populationSize):
            chromosome: str = ''
            for i in range(self.equationLength // 2):
                chromosome += str(random.choice(self.operands))
                chromosome += str(random.choice(self.operators))
            chromosome += str(random.choice(self.operands))
            chromosomes.append(chromosome)
        return chromosomes

    def findEquation(self):
        # Create a new generation of chromosomes, and make it better in every iteration
        while True:
            random.shuffle(self.population)
            fitnesses = []
            sortedChromosomes: list[tuple] = []
            for i in range(populationSize):
                # calculate the fitness of each chromosome
                fitness = calcFitness(self.population[i])

                # TODO return chromosome if a solution is found, else save the fitness in an array
                if fitness == 1:
                    return self.population[i]
                fitnesses.append(fitness)

                # TODO find the best chromosomes based on their fitnesses, and carry them directly to the next generation (optional)
                sortedChromosomes.append((fitness, self.population[i]))
            sortedChromosomes.sort()

            carriedChromosomes = []
            bestChromosome = []
            for i in range(0, int(populationSize * carryPercentage)):
                bestChromosome.append(sortedChromosomes[i][1])
                carriedChromosomes.append(bestChromosome[i])

            # A pool consisting of potential candidates for mating (crossover and mutation)
            matingPool = self.createMatingPool(fitnesses=fitnesses)

            # The pool consisting of chromosomes after crossover
            crossoverPool = self.createCrossoverPool(matingPool)

            # Delete the previous population
            self.population.clear()

            # Create the portion of population that is undergone crossover and mutation
            for i in range(populationSize - int(populationSize * carryPercentage)):
                self.population.append(self.mutate(crossoverPool[i]))

            # Add the prominent chromosomes directly to next generation
            self.population.extend(carriedChromosomes)

    def createMatingPool(self, fitnesses: list):
        # TODO make a brand new custom pool to accentuate prominent chromosomes (optional)
        fitnesses_sum = sum(fitnesses)
        probability_list = [fitness / fitnesses_sum for fitness in fitnesses]
        cumulative_probs_list = list(accumulate(probability_list))
        # TODO create the matingPool using custom pool created in the last step and return it
        matingPool = []
        for i in range(populationSize):
            rnd_num = random.uniform(0, 1)
            # if C[2] > R[1]> C[1]: then new_chromosome=chromosome[2] and etc.
            to_be_added = bisect.bisect_left(cumulative_probs_list, rnd_num)
            # print(rnd_num,to_be_added)
            matingPool.append(self.population[to_be_added])

        return matingPool

    def createCrossoverPool(self, matingPool):
        crossoverPool = []
        for i in range(len(matingPool)):
            if random.random() > crossoverProbability:
                # TODO don't perform crossover and add the chromosomes to the next generation directly to crossoverPool
                crossoverPool.append(matingPool[i])
            else:
                # TODO find 2 child chromosomes, crossover, and add the result to crossoverPool
                index1, index2 = 0, 0
                where_to_break = 0
                while index1 == index2:
                    index1 = random.randint(0, populationSize - 1)
                    index2 = random.randint(0, populationSize - 1)
                    where_to_break = random.randint(1, self.equationLength - 1)

                # second_half=equationLength-first_half
                new_chromosome1 = self.population[index1][:where_to_break] + self.population[index2][where_to_break:]
                new_chromosome2 = self.population[index2][:where_to_break] + self.population[index1][where_to_break:]
                crossoverPool.append(new_chromosome1)
                crossoverPool.append(new_chromosome2)

        return crossoverPool

    def mutate(self, chromosome):
        # mutate the input chromosome
        # mutate maximum 1/4 of chromosome
        for _ in range(self.equationLength // 4):
            where_to_mutate = random.randint(0, self.equationLength - 1)
            if where_to_mutate % 2 == 0:  # operand
                chromosome = chromosome[:where_to_mutate] + str(random.choice(self.operands)) + chromosome[
                                                                                                where_to_mutate + 1:]
            else:  # operator
                chromosome = chromosome[:where_to_mutate] + str(random.choice(self.operators)) + chromosome[
                                                                                                 where_to_mutate + 1:]

        return chromosome


#### Input

In [35]:
def read_input():
    operands = [1, 2, 3, 4, 5, 6, 7, 8]
    operators = ['+', '-', '*']
    equationLength = 21
    goalNumber = 18019

    return operands, operators, equationLength, goalNumber

In [69]:
if __name__ == '__main__':
    operands, operators, equationLength, goalNumber = read_input()
    equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber)
    equation = equationBuilder.findEquation()
    print(Fore.GREEN , equation  ,Fore.RESET)


 7-6+5*4+5*4*6*6*5*5-2 


## Questions

1. **`Very Large or Very Small Population`**
If the population is very small, the diversity decreases and it takes more time to calculate because the crossover function has lesser effect and it is the mutation funtion that plays the major role.\
On contrast, if the population is very large, the diversity increases but it takes more time to calculate because it calculate more states than it is needed.

2. **`Population Increases Effects on Algorithm`**
The diversity and number of chromosomes increases. But on the other hand, the calculations become heavier and data convergence may also be lost. 

3. **`Crossover & Mutation`**
Crossover takes 2 chromosomes and moves some of their genes to make a better combination by combining 2 genes. But Mutation directly changes the information of a chromosome so that we always have new chromosomes and do not stop in one generation of chromosomes.\
If the population is very small, Crossover will not be efficient and if the population is very large, the mutation alone will take a huge time.

4. **`Increase Time Efficiency for EP`**
Choosing the right value for the problem parameters and well defining the fitness function can be helpful.

5. **`How to Solve Getting Dtuck at Some Point and Chromosomes NOT Changing Problem`**
One of the reasons may be setting the mutationProbability a low number.This will cause the fitness not to increase and we will not reach the correct answer.\
We can solve this problem by increasing the mutationProbability
and run the algorithm several times.

6. **`In Case of a Problem Without an Answer, What Can Be Done`**
We can set a limit on how many times program should loop over the population and return the nearest answer to the goal answer.\
Another way is to set a time limit on the program.